In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q gdown && gdown --fuzzy https://drive.google.com/file/d/1HFIClmLVILw851oU3n-hAz3zXkcMZK-I/view      
!tar xf spark-3.1.2-bin-hadoop3.2_by_yk.tgz    
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("RDD").getOrCreate()
spark

In [4]:
rdd = spark.sparkContext.textFile("/content/text.txt")
rdd.collect()

['Bangladesh, country of South Asia, located in the delta of the Padma (Ganges [Ganga]) and Jamuna (Brahmaputra) rivers in the northeastern part of the Indian subcontinent.',
 'The riverine country of Bangladesh (“Land of the Bengals”) is one of the most densely populated countries in the world, and its people are predominantly Muslim. As the eastern portion of the historical region of Bengal, the area once formed, along with what is now the Indian state of West Bengal, the province of Bengal in British India. With the partition of India in 1947, it became the Pakistani province of East Bengal (later renamed East Pakistan), one of five provinces of Pakistan, separated from the other four by 1,100 miles (1,800 km) of Indian territory. In 1971 it became the independent country of Bangladesh, with its capital at Dhaka.Bangladesh is bordered by the Indian states of West Bengal to the west and north, Assam to the north, Meghalaya to the north and northeast, and Tripura and Mizoram to the ea

In [5]:
rdd2 = rdd.flatMap(lambda x: x.split(" "))
rdd2.take(24)

['Bangladesh,',
 'country',
 'of',
 'South',
 'Asia,',
 'located',
 'in',
 'the',
 'delta',
 'of',
 'the',
 'Padma',
 '(Ganges',
 '[Ganga])',
 'and',
 'Jamuna',
 '(Brahmaputra)',
 'rivers',
 'in',
 'the',
 'northeastern',
 'part',
 'of',
 'the']

In [6]:
rdd3 = rdd2.map(lambda x: (x,1))
rdd3.take(8)

[('Bangladesh,', 1),
 ('country', 1),
 ('of', 1),
 ('South', 1),
 ('Asia,', 1),
 ('located', 1),
 ('in', 1),
 ('the', 1)]

In [7]:
rdd4 = rdd3.reduceByKey(lambda a,b:a+b)
rdd4.take(8)

[('Bangladesh,', 2),
 ('country', 3),
 ('of', 38),
 ('South', 2),
 ('Asia,', 1),
 ('located', 1),
 ('in', 12),
 ('the', 57)]

In [8]:
rdd5 = rdd4.map(lambda x: (x[0], x[1])).sortByKey()
rdd5.take(8)

[('', 1),
 ('(1,800', 1),
 ('(160', 1),
 ('(3', 1),
 ('(9', 1),
 ('(Brahmaputra)', 2),
 ('(Burma).', 1),
 ('(Ganges', 2)]

In [9]:
#id: 193-15-2965
rdd6 = rdd5.repartition(5) 
print("Number of Partitions: "+ str(rdd6.getNumPartitions()))

Number of Partitions: 5


In [15]:
rdd6.saveAsTextFile("/content/Bangladesh")